<a href="https://colab.research.google.com/github/Skc69/MEMT680-/blob/main/MEM_T680_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium

df = pd.read_csv("..//content/WeatherEvents_Jan2016-Dec2021.csv.zip")
df.tail(3)



,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0
7479164,W-7577197,Snow,Light,2021-12-28 03:09:00,2021-12-28 03:19:00,0.01,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0


In [3]:
print(df.isna().sum())
print("\nHighest null ratio value of dataset column: 'ZipCode'")
df.ZipCode.isna().sum() / len(df)

EventId                  0
Type                     0
Severity                 0
StartTime(UTC)           0
EndTime(UTC)             0
Precipitation(in)        0
TimeZone                 0
AirportCode              0
LocationLat              0
LocationLng              0
City                 14563
County                   0
State                    0
ZipCode              59234
dtype: int64

Highest null ratio value of dataset column: 'ZipCode'


0.00791986806013773

In [4]:
df = df.dropna()

In [5]:
df.Type.value_counts()

Rain             4374978
Fog              1703947
Snow              967563
Cold              195920
Precipitation     128303
Storm              46494
Hail                2726
Name: Type, dtype: int64

In [6]:
df.describe()

,Precipitation(in),LocationLat,LocationLng,ZipCode
count,7.419931e+06,7.419931e+06,7.419931e+06,7.419931e+06
mean,9.558954e-02,3.873037e+01,-9.184624e+01,5.247809e+04
std,9.220315e-01,5.451146e+00,1.343660e+01,2.571490e+04
min,0.000000e+00,2.455710e+01,-1.245550e+02,1.022000e+03
25%,0.000000e+00,3.450710e+01,-9.771450e+01,3.131400e+04
50%,0.000000e+00,3.927330e+01,-8.973070e+01,5.401700e+04
75%,5.000000e-02,4.293800e+01,-8.193510e+01,7.352100e+04
max,1.104130e+03,4.894020e+01,-6.779280e+01,9.936200e+04


In [7]:
datetimeFormat = '%Y-%m-%d %H:%M:%S'
df['End']=pd.to_datetime(df['EndTime(UTC)'], format=datetimeFormat)
df['Start']=pd.to_datetime(df['StartTime(UTC)'], format=datetimeFormat)
df['Duration']=df['End']-df['Start']
df['Duration'] = df['Duration'].dt.total_seconds()
df['Duration'] = df['Duration'] / (60*60) # Converting to hours
df = df[(df['Duration']< 30*24) & (df['Duration'] > 0)] # More than 30 days or less than 0 hours discard.
df.tail(3)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,End,Start,Duration
7479162,W-7577195,Snow,Light,2021-12-28 00:53:00,2021-12-28 02:27:00,0.08,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2021-12-28 02:27:00,2021-12-28 00:53:00,1.566667
7479163,W-7577196,Snow,Moderate,2021-12-28 02:27:00,2021-12-28 03:09:00,0.09,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2021-12-28 03:09:00,2021-12-28 02:27:00,0.700000
7479164,W-7577197,Snow,Light,2021-12-28 03:09:00,2021-12-28 03:19:00,0.01,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2021-12-28 03:19:00,2021-12-28 03:09:00,0.166667


In [8]:
df.Duration.describe()


count    7.419580e+06
mean     1.307134e+00
std      4.353010e+00
min      1.666667e-02
25%      3.333333e-01
50%      6.666667e-01
75%      1.266667e+00
max      7.186667e+02
Name: Duration, dtype: float64

In [9]:
def print_duration_stats(df):
    print("Duration Summary by Hours")
    print("Count  ",df['Duration'].size)
    print("% Null ",df['Duration'].isnull().mean())
    print("Size   ",df['Duration'].unique().size)
    print("Min    ",df['Duration'].min())
    print("1stQrt ",df['Duration'].quantile(0.25))
    print("Mean   ",df['Duration'].mean())
    print("Median ",df['Duration'].median())
    print("3rdQrt ",df['Duration'].quantile(0.75))
    print("95%    ",df['Duration'].mean()+1.96*df['Duration'].std())
    print("Max    ",df['Duration'].max())
    print("Std    ",df['Duration'].std())
    
print_duration_stats(df)


Duration Summary by Hours
Count   7419580
% Null  0.0
Size    3960
Min     0.01666666666666667
1stQrt  0.3333333333333333
Mean    1.307133555178775
Median  0.6666666666666666
3rdQrt  1.2666666666666666
95%     9.839033458106105
Max     718.6666666666666
Std     4.35301015455476


In [10]:
df = df[df.Duration < 10]
print_duration_stats(df)

Duration Summary by Hours
Count   7340365
% Null  0.0
Size    599
Min     0.01666666666666667
1stQrt  0.3333333333333333
Mean    1.10481937887285
Median  0.6666666666666666
3rdQrt  1.2
95%     3.754416988932952
Max     9.983333333333333
Std     1.3518355153367867


In [11]:
df2 = df.groupby(['AirportCode','City','State', 
                  'LocationLat', 'LocationLng','Type', 'Severity']).agg({'Duration':['sum']}).reset_index()
df2.columns = pd.MultiIndex.from_tuples((("AirportCode", " "),("City", " "),
                                         ("State", " "), ("LocationLat", " "),
                                         ("LocationLng", " "), ("Type", " "), ("Severity", " "),
                                         ("Duration", " ")))
df2.columns = df2.columns.get_level_values(0)
df2['Duration'] = df2['Duration'] / (24*6*3.65) # Yearly Percentage
df2 = df2.sort_values(by='Duration')
df2.tail(3)

,AirportCode,City,State,LocationLat,LocationLng,Type,Severity,Duration
14580,KONP,South Beach,OR,44.5804,-124.0579,Rain,Light,12.366914
1889,KAST,Warrenton,OR,46.1569,-123.8825,Rain,Light,13.249239
19409,KUIL,Forks,WA,47.9375,-124.5550,Rain,Light,14.080257


In [12]:
south = ['AL', 'GA', 'FL', 'SC', 'NC', 'TN', 'MS', 'LA', 'AR', 'OK', 'TX']
atlantic = ['KY', 'VA', 'WV', 'MD', 'DE', 'PA', 'NJ']
new_england = ['NY', 'CT', 'RI', 'MA', 'VT', 'NH', 'ME']
midwest = ['OH', 'IN', 'MI', 'IL', 'WI', 'IA', 'MO', 'MN', 'NE', 'KS', 'SD', 'ND']
west = ['MT', 'WY', 'ID', 'NV', 'UT', 'CO', 'AZ', 'NM']
pacific = ['AK', 'WA', 'OR', 'CA', 'HI']
df2.loc[df2['State'].isin(south), 'Region'] = 'South'
df2.loc[df2['State'].isin(atlantic), 'Region'] = 'Atlantic'
df2.loc[df2['State'].isin(new_england), 'Region'] = 'New England'
df2.loc[df2['State'].isin(midwest), 'Region'] = 'Midwest'
df2.loc[df2['State'].isin(west), 'Region'] = 'West'
df2.loc[df2['State'].isin(pacific), 'Region'] = 'Pacific'
df2 = df2[['AirportCode', 'City', 'State', 'Region', 'LocationLat', 'LocationLng', 'Type', 'Severity', 'Duration']]
df2.tail(3)

,AirportCode,City,State,Region,LocationLat,LocationLng,Type,Severity,Duration
14580,KONP,South Beach,OR,Pacific,44.5804,-124.0579,Rain,Light,12.366914
1889,KAST,Warrenton,OR,Pacific,46.1569,-123.8825,Rain,Light,13.249239
19409,KUIL,Forks,WA,Pacific,47.9375,-124.5550,Rain,Light,14.080257


In [13]:
pre_flat = df.groupby(['AirportCode','City','State', 
                  'LocationLat', 'LocationLng','Type']).agg({'Duration':['sum']}).reset_index()
pre_flat.columns = pd.MultiIndex.from_tuples((("AirportCode", " "),("City", " "),
                                              ("State", " "), ("LocationLat", " "),
                                              ("LocationLng", " "), ("Type", " "),
                                              ("Duration", " ")))
pre_flat.columns = pre_flat.columns.get_level_values(0)
pre_flat['Duration'] = pre_flat['Duration'] / (24*6*3.65) # Yearly Percentage
pre_flat = pre_flat.sort_values(by='Duration')
pre_flat.tail(3)

,AirportCode,City,State,LocationLat,LocationLng,Type,Duration
277,K3TH,Thompson Falls-West End,MT,47.6000,-115.3667,Snow,14.716736
1087,KAST,Warrenton,OR,46.1569,-123.8825,Rain,14.862284
11163,KUIL,Forks,WA,47.9375,-124.5550,Rain,16.567383


In [14]:
df_flat = pre_flat.pivot_table(index='AirportCode', columns='Type', values=['Duration']).reset_index().fillna(0)
df_flat.columns = pd.MultiIndex.from_tuples(((' ', 'AirportCode'),(' ', 'Cold'),(' ', 'Fog'),
            (' ',  'Hail'),(' ', 'Precipitation'),(' ', 'Rain'),(' ', 'Snow'),(' ', 'Storm')))
df_flat.columns = df_flat.columns.get_level_values(1)
uniqueKey = df2[['AirportCode', 'City', 
                 'State', 'Region', 'LocationLat', 'LocationLng']].sort_values(by='AirportCode').drop_duplicates()
weather = pd.merge(df_flat, uniqueKey, how='inner', on='AirportCode')
weather.tail(3)

,AirportCode,Cold,Fog,Hail,Precipitation,Rain,Snow,Storm,City,State,Region,LocationLat,LocationLng
2051,KYNG,0.013128,2.591958,0.011035,0.095003,9.182078,4.489314,0.001459,Vienna,OH,Midwest,41.2544,-80.6739
2052,KZPH,0.381786,1.771943,0.000000,0.049467,4.465373,0.000000,0.001268,Zephyrhills,FL,South,28.2281,-82.1559
2053,KZZV,0.019945,1.861999,0.000000,0.070015,6.013413,1.695364,0.000000,Zanesville,OH,Midwest,39.9444,-81.8921


In [15]:
fig_sum = px.histogram(df2, x='Type', y= 'Duration',  histfunc = 'avg', color='Severity',
                      title = 'Fig 1. Nation Wide Weather Events Duration')
fig_sum.update_xaxes(categoryorder='mean descending')
fig_sum.update_yaxes(title_text='Mean of Duration % per Year')
fig_sum.update_layout(height=750, width=1000)
fig_sum.show()

In [16]:
fig_state=make_subplots(rows=7, cols=1, shared_yaxes=False, vertical_spacing=0.05)

fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Rain'], name='Rain', histfunc ='avg'),1,1)
# fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Fog'], name='Fog', histfunc ='avg'),2,1)
# fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Snow'], name='Snow', histfunc ='avg'),3,1)
# fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Cold'], name='Cold', histfunc ='avg'),4,1)
# fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Precipitation'], name='Precipitation', histfunc ='avg'),5,1)
# fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Storm'], name='Storm', histfunc ='avg'),6,1)
# fig_state.add_trace(go.Histogram(x=weather['Region'], y=weather['Hail'], name='Hail', histfunc ='avg'),7,1)

fig_state['layout']['xaxis7'].update(title="Region")
fig_state['layout']['yaxis4'].update(title="Mean Duration % per Year")
fig_state.update_xaxes(categoryorder='mean descending')
fig_state.update_layout( title_text="Fig 2. Region Wide Weather Events Duration Distribution")
# fig_state.update_yaxes(ticklabelstep=2)
fig_state.show()

In [17]:
fig_state=make_subplots(rows=7, cols=1, shared_yaxes=False, vertical_spacing=0.05)

fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Rain'], name='Rain', histfunc ='avg'),1,1)
# fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Fog'], name='Fog', histfunc ='avg'),2,1)
# fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Snow'], name='Snow', histfunc ='avg'),3,1)
# fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Cold'], name='Cold', histfunc ='avg'),4,1)
# fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Precipitation'], name='Precipitation', histfunc ='avg'),5,1)
# fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Storm'], name='Storm', histfunc ='avg'),6,1)
# fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Hail'], name='Hail', histfunc ='avg'),7,1)

fig_state['layout']['xaxis7'].update(title="State")
fig_state['layout']['yaxis4'].update(title="Mean Duration % per Year")
fig_state.update_xaxes(categoryorder='mean descending')
fig_state.update_layout( title_text="Fig 3. State Wide Weather Events Duration Distribution")
fig_state.update_xaxes(tickangle=0)
# fig_state.update_yaxes(ticklabelstep=2)
fig_state.show()

In [18]:
fig_rain = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Rain",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense', 
                      scope="usa",
                      title ='Fig 5. City Wide Rainy Days Percentage Each Year from 2016 to 2021')
fig_rain.show()

In [19]:
from sklearn.cluster import KMeans

X = df_flat.drop(['AirportCode','Cold', 'Hail'], axis=1)
X.tail(3)

,Fog,Precipitation,Rain,Snow,Storm
2051,2.591958,0.095003,9.182078,4.489314,0.001459
2052,1.771943,0.049467,4.465373,0.000000,0.001268
2053,1.861999,0.070015,6.013413,1.695364,0.000000


In [20]:
from sklearn.cluster import KMeans
distortions = []

K = range(1,20)
for k in K:
    kmean = KMeans(n_clusters=k, random_state=0, n_init = 50, max_iter = 500)
    kmean.fit(X)
    distortions.append(kmean.inertia_)
    
fig_kmean = px.scatter(x=K, y=distortions, title='Fig 10. The Elbow Method')
fig_kmean.update_xaxes(title_text='k')
fig_kmean.update_yaxes(title_text='distortion')
fig_kmean.show()

In [23]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

df_flat['Cluster'] = (kmeans.labels_).astype(str)
df_cluster = pd.merge(df_flat[['AirportCode','Cluster']], weather.drop(['Cold','Hail'], axis=1), 
                      how='inner', on='AirportCode')
df_cluster.tail(3)

,AirportCode,Cluster,Fog,Precipitation,Rain,Snow,Storm,City,State,Region,LocationLat,LocationLng
2051,KYNG,1,2.591958,0.095003,9.182078,4.489314,0.001459,Vienna,OH,Midwest,41.2544,-80.6739
2052,KZPH,0,1.771943,0.049467,4.465373,0.000000,0.001268,Zephyrhills,FL,South,28.2281,-82.1559
2053,KZZV,1,1.861999,0.070015,6.013413,1.695364,0.000000,Zanesville,OH,Midwest,39.9444,-81.8921


In [24]:
fig_cluster = px.scatter_geo(df_cluster, lat='LocationLat', lon='LocationLng',
                      hover_name=weather['City'] + ', ' + weather['State'],
                      scope="usa",
                      color_discrete_sequence =['#AB63FA', '#EF553B', '#00CC96','#636EFA'],
                      color = 'Cluster',
                      title ='Fig 11. City Wide Weather Cluster Distribution')
fig_cluster.show()

In [25]:
df_cluster2 = df_cluster.groupby(['State','Cluster']).agg({'Cluster':['count']}).reset_index()
df_cluster2.columns= pd.MultiIndex.from_tuples((("State", " "),("Cluster", " "),("Count", " ")))
df_cluster2.columns = df_cluster2.columns.get_level_values(0)

df_locS = df_cluster[['State','Cluster','LocationLat', 'LocationLng']]
df_locS1 = df_locS.groupby(['State','Cluster']).agg({'LocationLat':'mean'}).reset_index()
df_locS2 = df_locS.groupby(['State','Cluster']).agg({'LocationLng':'mean'}).reset_index()
df_locS3 = pd.merge(df_locS1,df_locS2, how='inner', on=['State','Cluster'])

df_clusterS = pd.merge(df_locS3,df_cluster2, how='inner', on=['State','Cluster'])
df_clusterS.tail(3) # State with each cluster count location

,State,Cluster,LocationLat,LocationLng,Count
131,WV,1,38.709213,-80.693020,15
132,WY,2,42.724793,-107.109307,29
133,WY,3,42.674508,-107.759944,25


In [27]:
fig_clusterS = px.scatter_geo(df_clusterS, lat='LocationLat', lon='LocationLng', 
                     color='Cluster',
                     size='Count',
                     color_discrete_sequence=['#636EFA', '#AB63FA', '#EF553B','#00CC96'],
                     hover_name='State',
                     scope="usa",
                     title = 'Fig 12. State Wide Weather Cluster Distribution')
fig_clusterS.show()